**Review**

Hi, my name is Dmitry and I will be reviewing your project.
  
You can find my comments in colored markdown cells:
  
<div class="alert alert-success">
  If everything is done successfully.
</div>
  
<div class="alert alert-warning">
  If I have some (optional) suggestions, or questions to think about, or general comments.
</div>
  
<div class="alert alert-danger">
  If a section requires some corrections. Work can't be accepted with red comments.
</div>
  
Please don't remove my comments, as it will make further review iterations much harder for me.
  
Feel free to reply to my comments or ask questions using the following template:
  
<div class="alert alert-info">
  For your comments and questions.
</div>
  
First of all, thank you for turning in the project! You did an excellent job! The project is accepted. Keep up the good work on the next sprint!

Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

## Data preparation

To begin, we will first perform the following on the data:

- Load the necessary libraries
- Import the file into Dataframes
- Display the data
- View the info
- Adjust column names
- Describe the data
- Check for nulls and fill in, if possible
- Check the unique values
- Check for duplicates
- Adjust data set for unnecessary columns
- Fix any data types
- Encode the data
- Scale the data

In [1]:
# Load the necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MaxAbsScaler
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.metrics import mean_squared_error, mean_absolute_error,roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from catboost import CatBoostClassifier, CatBoostRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [2]:
# Import the file into Dataframes
try:
    df_car = pd.read_csv('/datasets/car_data.csv', sep=',')

except FileNotFoundError:
    df_car = pd.read_csv('../datasets/car_data.csv', sep=',')

**Features**

- DateCrawled — date profile was downloaded from the database
- VehicleType — vehicle body type
- RegistrationYear — vehicle registration year
- Gearbox — gearbox type
- Power — power (hp)
- Model — vehicle model
- Mileage — mileage (measured in km due to dataset's regional specifics)
- RegistrationMonth — vehicle registration month
- FuelType — fuel type
- Brand — vehicle brand
- NotRepaired — vehicle repaired or not
- DateCreated — date of profile creation
- NumberOfPictures — number of vehicle pictures
- PostalCode — postal code of profile owner (user)
- LastSeen — date of the last activity of the user

**Target**

- Price — price (Euro)

In [3]:
display(df_car)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


In [4]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

<div class="alert alert-success">
<b>Reviewer's comment</b>

The data was loaded and inspected

</div>

In [5]:
# Change column names
df_car.columns = df_car.columns.str.lower()
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354369 non-null  object
 1   price              354369 non-null  int64 
 2   vehicletype        316879 non-null  object
 3   registrationyear   354369 non-null  int64 
 4   gearbox            334536 non-null  object
 5   power              354369 non-null  int64 
 6   model              334664 non-null  object
 7   mileage            354369 non-null  int64 
 8   registrationmonth  354369 non-null  int64 
 9   fueltype           321474 non-null  object
 10  brand              354369 non-null  object
 11  notrepaired        283215 non-null  object
 12  datecreated        354369 non-null  object
 13  numberofpictures   354369 non-null  int64 
 14  postalcode         354369 non-null  int64 
 15  lastseen           354369 non-null  object
dtypes: int64(7), object(

In [6]:
# Descriptive Statistics
df_car.describe()

,price,registrationyear,power,mileage,registrationmonth,numberofpictures,postalcode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [7]:
#df_car[df_car['price'] == 0]  # 10772 rows
len(df_car[(df_car['power'] > 10000)])  # (df_car['power'] < 20) | 

44

In [8]:
df_car['registrationyear'].describe()

count    354369.000000
mean       2004.234448
std          90.227958
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: registrationyear, dtype: float64

In [9]:
df_car['registrationyear'].value_counts(ascending=False, dropna=False)

2000    24490
1999    22728
2005    22109
2001    20124
2006    19900
        ...  
4100        1
1200        1
5300        1
8888        1
2290        1
Name: registrationyear, Length: 151, dtype: int64

In [10]:
no_year = df_car[(df_car['registrationyear'] < 1900) | (df_car['registrationyear'] > 2024)]
display(no_year) #.index)

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
622,16/03/2016 16:55,0,NaN,1111,NaN,0,NaN,5000,0,NaN,opel,NaN,16/03/2016 00:00,0,44628,20/03/2016 16:44
12946,29/03/2016 18:39,49,NaN,5000,NaN,0,golf,5000,12,NaN,volkswagen,NaN,29/03/2016 00:00,0,74523,06/04/2016 04:16
15147,14/03/2016 00:52,0,NaN,9999,NaN,0,NaN,10000,0,NaN,sonstige_autos,NaN,13/03/2016 00:00,0,32689,21/03/2016 23:46
15870,02/04/2016 11:55,1700,NaN,3200,NaN,0,NaN,5000,0,NaN,sonstige_autos,NaN,02/04/2016 00:00,0,33649,06/04/2016 09:46
16062,29/03/2016 23:42,190,NaN,1000,NaN,0,mondeo,5000,0,NaN,ford,NaN,29/03/2016 00:00,0,47166,06/04/2016 10:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340548,02/04/2016 17:44,0,NaN,3500,manual,75,NaN,5000,3,petrol,sonstige_autos,NaN,02/04/2016 00:00,0,96465,04/04/2016 15:17
340759,04/04/2016 23:55,700,NaN,1600,manual,1600,a3,150000,4,petrol,audi,no,04/04/2016 00:00,0,86343,05/04/2016 06:44
341791,28/03/2016 17:37,1,NaN,3000,NaN,0,zafira,5000,0,NaN,opel,NaN,28/03/2016 00:00,0,26624,02/04/2016 22:17
348830,22/03/2016 00:38,1,NaN,1000,NaN,1000,NaN,150000,0,NaN,sonstige_autos,NaN,21/03/2016 00:00,0,41472,05/04/2016 14:18


The 171 values for the registrationyear column don't make sense, so it would be best to remove them.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Makes sense!

</div>

In [11]:
df_car.drop(no_year.index, inplace=True)
df_car.reset_index(drop=True, inplace=True)
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354198 entries, 0 to 354197
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354198 non-null  object
 1   price              354198 non-null  int64 
 2   vehicletype        316879 non-null  object
 3   registrationyear   354198 non-null  int64 
 4   gearbox            334503 non-null  object
 5   power              354198 non-null  int64 
 6   model              334568 non-null  object
 7   mileage            354198 non-null  int64 
 8   registrationmonth  354198 non-null  int64 
 9   fueltype           321431 non-null  object
 10  brand              354198 non-null  object
 11  notrepaired        283191 non-null  object
 12  datecreated        354198 non-null  object
 13  numberofpictures   354198 non-null  int64 
 14  postalcode         354198 non-null  int64 
 15  lastseen           354198 non-null  object
dtypes: int64(7), object(

### Nulls

We will need to decide what to do with the nulls.

Our strategy to fill in nulls:
- nan to 'other' for multiple columns
- combine petrol and gasoline as the same category.

In [12]:
df_car.isna().sum()

datecrawled              0
price                    0
vehicletype          37319
registrationyear         0
gearbox              19695
power                    0
model                19630
mileage                  0
registrationmonth        0
fueltype             32767
brand                    0
notrepaired          71007
datecreated              0
numberofpictures         0
postalcode               0
lastseen                 0
dtype: int64

In [13]:
df_car[df_car['model'].isna()]

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
59,29/03/2016 15:48,1,suv,1994,manual,286,NaN,150000,11,NaN,sonstige_autos,NaN,29/03/2016 00:00,0,53721,06/04/2016 01:44
81,03/04/2016 12:56,350,small,1997,manual,54,NaN,150000,3,NaN,fiat,yes,03/04/2016 00:00,0,45665,05/04/2016 11:47
115,20/03/2016 18:53,0,small,1999,NaN,0,NaN,5000,0,petrol,volkswagen,NaN,20/03/2016 00:00,0,37520,07/04/2016 02:45
135,27/03/2016 20:51,1450,sedan,1992,manual,136,NaN,150000,0,NaN,audi,no,27/03/2016 00:00,0,38709,05/04/2016 20:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354074,07/03/2016 16:37,560,small,2001,auto,170,NaN,90000,0,petrol,fiat,yes,07/03/2016 00:00,0,55743,12/03/2016 21:45
354150,15/03/2016 13:52,9400,wagon,2007,manual,200,NaN,150000,4,gasoline,sonstige_autos,yes,15/03/2016 00:00,0,91186,17/03/2016 08:16
354167,31/03/2016 19:52,180,NaN,1995,NaN,0,NaN,125000,3,petrol,opel,NaN,31/03/2016 00:00,0,41470,06/04/2016 14:18
354180,11/03/2016 23:40,1900,NaN,2000,manual,110,NaN,150000,7,NaN,volkswagen,no,11/03/2016 00:00,0,87700,12/03/2016 14:16


In [14]:
# Check Duplicates
print(df_car[df_car.duplicated()])

             datecrawled  price  vehicletype  registrationyear gearbox  power  \
14264   21/03/2016 19:06   5999        small              2009  manual     80   
27556   23/03/2016 10:38  12200          bus              2011  manual    125   
31584   03/04/2016 20:41   4950        wagon              2003    auto    170   
33123   07/03/2016 20:45  10900  convertible              2005    auto    163   
43637   13/03/2016 20:48   4200        sedan              2003  manual    105   
...                  ...    ...          ...               ...     ...    ...   
349539  03/04/2016 20:52    700        small              1999  manual     60   
351385  26/03/2016 16:54   3150          bus              2003  manual     86   
352213  15/03/2016 21:54   5900        wagon              2006  manual    129   
352886  05/03/2016 14:16   9500        small              2013  manual    105   
353556  20/03/2016 17:56      1          NaN              2000     NaN      0   

           model  mileage  

We will choose not to delete the duplicates because it is possible that the same car is on sale multiple times.

<div class="alert alert-warning">
<b>Reviewer's comment</b>

In that case `datecreated` should be different, right?

</div>

In [15]:
# Uniqueness
print(df_car['vehicletype'].unique())
print(df_car['gearbox'].unique())
print(df_car['model'].unique())
print(df_car['model'].nunique())
print(df_car['fueltype'].unique())
print(df_car['brand'].unique())
print(df_car['notrepaired'].unique())
print(df_car['numberofpictures'].unique())

[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']
['manual' 'auto' nan]
['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz' 'beetle' 'c1' 'rio' 'touareg'
 'logan' 'spider' 'cuore' 's_max' 'a2' 'galaxy' 'c3

In [16]:
pd.Series(df_car['model'].unique()).sort_values()

59         100
232        145
58         147
41         156
130        159
        ...   
92        yeti
150    ypsilon
60     z_reihe
74      zafira
1          NaN
Length: 251, dtype: object

In [17]:
# Drop unnecesary columns since they shouldn't help predict the price

In [18]:
col_drop = ['datecrawled', 'datecreated', 'numberofpictures', 'lastseen', 'postalcode']
df_car.drop(col_drop, axis=1, inplace=True)
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354198 entries, 0 to 354197
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              354198 non-null  int64 
 1   vehicletype        316879 non-null  object
 2   registrationyear   354198 non-null  int64 
 3   gearbox            334503 non-null  object
 4   power              354198 non-null  int64 
 5   model              334568 non-null  object
 6   mileage            354198 non-null  int64 
 7   registrationmonth  354198 non-null  int64 
 8   fueltype           321431 non-null  object
 9   brand              354198 non-null  object
 10  notrepaired        283191 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


<div class="alert alert-success">
<b>Reviewer's comment</b>

Makes sense!

</div>

In [19]:
df_car['vehicletype'].value_counts(ascending=False, dropna=False)

sedan          91457
small          79831
wagon          65166
NaN            37319
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: vehicletype, dtype: int64

In [20]:
df_car[['vehicletype', 'gearbox', 'model', 'fueltype', 'notrepaired']] = \
    df_car[['vehicletype', 'gearbox', 'model', 'fueltype', 'notrepaired']].fillna('other')

In [21]:
df_car.isna().sum()

price                0
vehicletype          0
registrationyear     0
gearbox              0
power                0
model                0
mileage              0
registrationmonth    0
fueltype             0
brand                0
notrepaired          0
dtype: int64

In [22]:
df_car['fueltype'] = df_car['fueltype'].replace('gasoline', 'petrol')
df_car['fueltype'].value_counts(ascending=False, dropna=False)

petrol      315031
other        32970
lpg           5310
cng            564
hybrid         233
electric        90
Name: fueltype, dtype: int64

<div class="alert alert-success">
<b>Reviewer's comment</b>

Missing values were dealt with reasonably

</div>

### Encoding and Scaling

- OHE for Linear Regression and XGBoost
- Ordinal/Label for CatBoost, LGBM, DTR, and RFR

Since some models require different encoding, we will make a data set with One Hot Encoding (OHE) and another one with ordinal encoding for the categorical columns.

After splitting the data into its features and targets, we will scale the data for the OHE data set.

In [23]:
# Encoding with OHE
ohe_cols = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']

In [24]:
%%timeit
df_car

30.3 ns ± 0.979 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [25]:
df_car_ohe = pd.get_dummies(df_car, columns=ohe_cols, drop_first=True)
df_car_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354198 entries, 0 to 354197
Columns: 309 entries, price to notrepaired_yes
dtypes: int64(5), uint8(304)
memory usage: 116.2 MB


In [26]:
for i in df_car_ohe.columns:
    print(i)

price
registrationyear
power
mileage
registrationmonth
vehicletype_convertible
vehicletype_coupe
vehicletype_other
vehicletype_sedan
vehicletype_small
vehicletype_suv
vehicletype_wagon
gearbox_manual
gearbox_other
model_145
model_147
model_156
model_159
model_1_reihe
model_1er
model_200
model_2_reihe
model_300c
model_3_reihe
model_3er
model_4_reihe
model_500
model_5_reihe
model_5er
model_601
model_6_reihe
model_6er
model_7er
model_80
model_850
model_90
model_900
model_9000
model_911
model_a1
model_a2
model_a3
model_a4
model_a5
model_a6
model_a8
model_a_klasse
model_accord
model_agila
model_alhambra
model_almera
model_altea
model_amarok
model_antara
model_arosa
model_astra
model_auris
model_avensis
model_aveo
model_aygo
model_b_klasse
model_b_max
model_beetle
model_berlingo
model_bora
model_boxster
model_bravo
model_c1
model_c2
model_c3
model_c4
model_c5
model_c_klasse
model_c_max
model_c_reihe
model_caddy
model_calibra
model_captiva
model_carisma
model_carnival
model_cayenne
model_cc
m

In [27]:
df_car_ohe.loc[20, 'price': 'mileage']

price                10400
registrationyear      2009
power                  160
mileage             100000
Name: 20, dtype: int64

In [28]:
features_ohe = df_car_ohe.drop(['price'], axis=1)
target_ohe = df_car_ohe['price']
print(features_ohe.shape)
print(target_ohe.shape)

(354198, 308)
(354198,)


In [29]:
features_train_ohe, features_test_ohe, target_train_ohe, target_test_ohe = train_test_split(
    features_ohe, target_ohe, test_size=0.3, random_state=12345
)

In [30]:
# Scaling OHE data set
scaler = MaxAbsScaler()
scaler.fit(features_train_ohe)
features_train = scaler.transform(features_train_ohe)
print(features_train_ohe)
features_test = scaler.transform(features_test_ohe)
print(features_test_ohe)

        registrationyear  power  mileage  registrationmonth  \
54123               2000     54   100000                  6   
216907              1999     64   150000                  1   
10852               2000      0   150000                  0   
325823              2001    129   150000                  9   
331331              2002    170   150000                  1   
...                  ...    ...      ...                ...   
47873               1998    101    90000                  9   
86398               1995    245    70000                 11   
347556              2018      0    50000                  0   
77285               2003    136    90000                  3   
217570              2004     60   150000                  6   

        vehicletype_convertible  vehicletype_coupe  vehicletype_other  \
54123                         0                  0                  0   
216907                        0                  0                  0   
10852                   

In [31]:
# Encoding with OrdinalEncoder
enc = OrdinalEncoder()
df_ord = df_car.copy()
df_ord[['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']] = \
    enc.fit_transform(df_car[['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']])
df_ord.sample(10)

,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired
161548,5300,4.0,2003,1.0,143,59.0,150000,12,5.0,20.0,0.0
8983,7450,0.0,2011,2.0,0,166.0,100000,12,4.0,5.0,1.0
353330,11300,4.0,1988,1.0,29,166.0,150000,3,5.0,5.0,0.0
114581,16900,6.0,2009,1.0,150,219.0,80000,10,5.0,38.0,0.0
26120,7200,2.0,2013,0.0,71,106.0,30000,3,5.0,32.0,1.0
340563,1900,0.0,2001,1.0,116,166.0,150000,5,5.0,5.0,0.0
187778,8990,7.0,2006,0.0,177,15.0,150000,12,5.0,2.0,0.0
87592,5350,1.0,2004,0.0,82,188.0,100000,8,5.0,32.0,0.0
174716,8300,4.0,2006,0.0,256,29.0,90000,7,5.0,1.0,0.0
325604,700,5.0,1999,1.0,60,173.0,150000,9,5.0,38.0,0.0


In [32]:
features_ord = df_ord.drop(['price'], axis=1)
target_ord = df_ord['price']
#print(features)
#print(target)
features_train_ord, features_test_ord, target_train_ord, target_test_ord = train_test_split(
    features_ord, target_ord, test_size=0.3, random_state=12345
)
features_train_ord.shape

(247938, 10)

<div class="alert alert-success">
<b>Reviewer's comment</b>

Caterogical features were encoded, the data was split into train and test, scaling was applied correctly

</div>

## Model training

Train different models with various hyperparameters (You should make at least two different models, but more is better. Remember, various implementations of gradient boosting don't count as different models.) The main point of this step is to compare gradient boosting methods with random forest, decision tree, and linear regression.

- Use the RMSE metric to evaluate the models.
- Linear regression is not very good for hyperparameter tuning, but it is perfect for doing a sanity check of other methods. If gradient boosting performs worse than linear regression, something definitely went wrong.
- On your own, work with the LightGBM library and use its tools to build gradient boosting models.
- Ideally, your project should include linear regression for a sanity check, a tree-based algorithm with hyperparameter tuning (preferably, random forrest), LightGBM with hyperparameter tuning (try a couple of sets), and CatBoost and XGBoost with hyperparameter tuning (optional).
- Take note of the encoding of categorical features for simple algorithms. LightGBM and CatBoost have their implementation, but XGBoost requires OHE.
- You can use a special command to find the cell code runtime in Jupyter Notebook. Find that command.
- Since the training of a gradient boosting model can take a long time, change only a few model parameters.

### Sanity Check

Sanity check using LinearRegression

In [34]:
%%timeit -n 3 -r 2
# LinearRegression
model_lr = LinearRegression()
model_lr.fit(features_train_ohe, target_train_ohe)
predictions_lr = model_lr.predict(features_test_ohe)
rmse_lr = mean_squared_error(target_test_ohe, predictions_lr)**0.5
print('RMSE:', rmse_lr)
# RMSE: 2949.2

RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
RMSE: 2949.2005334050336
19.7 s ± 54 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


### DecisionTreeRegressor and RandomForestRegressor

In [35]:
def model_optimizer(model, parameters, scoring_metric='neg_mean_squared_error', cv=5):

    grid_search = GridSearchCV(model, parameters, cv=cv, scoring=scoring_metric)
    grid_search.fit(features_train_ord, target_train_ord)

    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    return best_params, best_score

In [36]:
dt_params = {'max_depth': list(range(9, 18)),
             'random_state': [12345]
            }
dt_model = DecisionTreeRegressor()

best_params, best_score = model_optimizer(dt_model, dt_params, scoring_metric='neg_mean_squared_error')
print(best_params)
print(best_score)
# {'max_depth': 13, 'random_state': 12345}
# -4403431.3

{'max_depth': 13, 'random_state': 12345}
-4403431.312707396


In [38]:
dt_params_rfr = {'max_depth': list(range(14, 17)),
                 'n_estimators': list(range(40, 61, 10)),
                 'random_state': [12345]
                }
dt_model_rfr = RandomForestRegressor()

best_params_rfr, best_score_rfr = model_optimizer(dt_model_rfr, dt_params_rfr, scoring_metric='neg_mean_squared_error')
print(best_params_rfr)
print(best_score_rfr)
# {'max_depth': 15, 'n_estimators': 50, 'random_state': 12345}
# -3233139.9298187154

{'max_depth': 16, 'n_estimators': 60, 'random_state': 12345}
-3233139.9298187154


In [42]:
%%timeit -n 3 -r 2
# DecisionTreeRegressor
model_dtr = DecisionTreeRegressor(random_state=12345, max_depth=13)
model_dtr.fit(features_train_ord, target_train_ord)
predictions_dtr = model_dtr.predict(features_test_ord) 
result_dtr = mean_squared_error(target_test_ord, predictions_dtr)**0.5
print('RMSE:', result_dtr) # with ohe: 2051.7; with ord: 2066.2

RMSE: 2066.1774260220623
RMSE: 2066.1774260220623
RMSE: 2066.1774260220623
RMSE: 2066.1774260220623
RMSE: 2066.1774260220623
RMSE: 2066.1774260220623
1.05 s ± 280 µs per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [43]:
%%timeit -n 2 -r 2
# RandomForestRegressor
model_rfr = RandomForestRegressor(random_state=12345, n_estimators=60, max_depth=16) # initialize model constructor with parameters random_state=12345 and n_estimators=est
model_rfr.fit(features_train_ord, target_train_ord) # train model on training set
predictions_rfr = model_rfr.predict(features_test_ord) # get model predictions on validation set
result_rfr = mean_squared_error(target_test_ord, predictions_rfr)**0.5
print('RMSE:', result_rfr) # with ohe: 2039.2; with ord: 1777.8

RMSE: 1777.8064546975368
RMSE: 1777.8064546975368
47.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


### XGBoost
Requires OHE

In [56]:
%%timeit -n 1 -r 1
# XGBRegressor
xgbr = XGBRegressor(objective='reg:squarederror')
xgbr.fit(features_train_ohe, target_train_ohe)
 
predictions_xgbr = xgbr.predict(features_test_ohe)
rmse_xgbr = mean_squared_error(target_test_ohe, predictions_xgbr)**0.5
print("RMSE:", rmse_xgbr)  # RMSE: 1814.5

RMSE: 1814.529054287198
7min 48s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### LightGBM and CatBoost

Don't need OHE, but Label/Ordinal Code columns

In [ ]:
df_ord.sample(8)

In [61]:
%%timeit -n 2 -r 2
# Catboost
model_cbr = CatBoostRegressor(loss_function="RMSE", iterations=200, learning_rate=0.5, random_seed=12345) # learning_rate=0.4, 
model_cbr.fit(features_train_ord, target_train_ord, verbose=10)
pred_cbr = model_cbr.predict(features_test_ord)
mse = mean_squared_error(target_test_ord, pred_cbr)**0.5
print(mse)  # 1901.3; 1839.1 @ 100 iter; 1810.4 @ 150 iter; 1795.8 @ 200 iter, 0.5 l_rate

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 3280.0642722	total: 68.9ms	remaining: 13.7s
10:	learn: 2098.8142228	total: 647ms	remaining: 11.1s
20:	learn: 1985.8926504	total: 1.21s	remaining: 10.3s
30:	learn: 1936.3709637	total: 1.73s	remaining: 9.45s
40:	learn: 1903.8093180	total: 2.27s	remaining: 8.79s
50:	learn: 1878.2110027	total: 2.79s	remaining: 8.14s
60:	learn: 1857.3784779	total: 3.31s	remaining: 7.54s
70:	learn: 1839.5288116	total: 3.86s	remaining: 7.02s
80:	learn: 1824.9389238	total: 4.4s	remaining: 6.46s
90:	learn: 1809.0421086	total: 4.92s	remaining: 5.89s
100:	learn: 1796.6203385	total: 5.42s	remaining: 5.32s
110:	learn: 1786.5578223	total: 5.94s	remaining: 4.76s
120:	learn: 1776.3788969	total: 6.46s	remaining: 4.21s
130:	learn: 1766.8328918	total: 6.99s	remaining: 3.68s
140:	learn: 1757.5099047	total: 7.5s	remaining: 3.14s
150:	learn: 1749.0759181	total: 8s	remaining: 2.59s
160:	learn: 1741.0172001	total: 8.47s	remaining: 2.05s
170:	learn: 1734.5539080	total: 9s	remaining: 1.52s
180:	learn: 1727.3715396	tot

In [59]:
%%timeit -n 3 -r 2
# LGBMRegressor
model_lgbm = LGBMRegressor(metric='rmse', n_estimators=200)
model_lgbm.fit(features_train_ord, target_train_ord)
pred_lgbm = model_lgbm.predict(features_test_ord)
mse_lgbm = mean_squared_error(target_test_ord, pred_lgbm)**0.5
print(mse_lgbm)  # 1863.1; 1830.3 @ 150 n_est; 1809 @ 200 n_est

1809.4667391218184
1809.4667391218184
1809.4667391218184
1809.4667391218184
1809.4667391218184
1809.4667391218184
7.84 s ± 170 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


<div class="alert alert-success">
<b>Reviewer's comment</b>

Great, you tried a few different models, did some hyperparameter tuning using cross-validation and compared the final models using the test set

</div>

## Model analysis

Analyze the speed and quality of the models.

RandomForestRegressor is accurate with an RMSE of 1777.8, but slow (41.7 s ± 972 ms).

XGBRegressor comes in second in terms of the RMSE with , but is slowest model by far (7min 37s ± 1.59 s).

In order of the RMSE values, Catboost (10.8 s ± 25.7 ms), LGBMRegressor (8.5 s ± 2.01 s), DecisionTreeRegressor (1.05 s ± 280 µs per loop), and lastly, LinearRegression (19.7 s ± 54 ms) round out the rest of the accuracies, but were much faster than the the RandomForest and XGBoost. All the models were able to beat our sanity check with the LinearRegression model in terms of RMSE. However, the speed at which RandomForest and XGBoost models performed was much slower than the LinearRegression's baseline speed.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Very good, the analysis takes into account both speed and quality of predictions of the models

</div>

## Conclusion

The model that Rusty Bargain should use for quality, speed, and time is CatBoostRegressor with 200 iterations and a learning rate of 0.5. This model did well overall in the three qualities that Rusty Bargain required. It came in a close second in terms of the RMSE value behind the RandomForest model, which makes it's quality high. It also ranked as the third fastest model to train and make it's predictions, whereas the RandomForest model was not able to beat our benchmark time set by the LinearRegression model.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Alright!

</div>

# Checklist

Type 'x' to check. Then press Shift+Enter.

- [x]  Jupyter Notebook is open
- [ ]  Code is error free
- [ ]  The cells with the code have been arranged in order of execution
- [ ]  The data has been downloaded and prepared
- [ ]  The models have been trained
- [ ]  The analysis of speed and quality of the models has been performed